
# Exercice 2 (Partie B): Contre-factuels (What-If)

## Objectif
Implémenter l'approche What-If pour générer des contre-factuels pour un classifieur binaire.
L'idée est de trouver l'observation existante la plus proche dans le dataset qui a une prédiction différente de celle de l'instance d'intérêt.

Nous utilisons le fichier `whatif.py`.


In [1]:

import sys
sys.path.append(".")
import numpy as np
import pandas as pd
import gower
from sklearn import ensemble
from Dataset.dataset import Dataset
import whatif

# Setup Dataset (Wheat Seeds converted to Binary)
dataset = Dataset("wheat_seeds", range(0, 7), [7], normalize=True, categorical=True)

# Create binary classification task (Class 0 vs Rest)
y = dataset.y.copy()
# Remap: Class 0 remains 0, others become 1? Or just make it binary.
# The previous code did: y[y == 0] = 1. Let's check original classes.
# Original classes: 1, 2, 3. 
# Dataset.y usually straight from CSV. 
# If dataset is Wheat Seeds, types are 1, 2, 3.
# The code `y[y == 0] = 1` implies there's 0s.
# Let's trust the provided snippet logic, but maybe adjust for Wheat Seeds which is 1,2,3.
# We will use dataset.y directly and see.

# Re-read y to be safe
y = dataset.y.ravel()
# Make it binary: Class 1 vs not Class 1
y_binary = np.where(y == 1, 1, 0)

print("Classes distribution (Binary):", np.unique(y_binary, return_counts=True))

# Reserve first row for interest
X = dataset.X
x_interest = X[0,:].reshape(1, -1)

# Training set = Rest
X_train = np.delete(X, (0), axis=0)
y_train = np.delete(y_binary, (0), axis=0)

# Train Random Forest
model = ensemble.RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Probe x_interest
print("x_interest features:", x_interest)
print("Prediction x_interest:", model.predict(x_interest))


Classes distribution (Binary): (array([0, 1]), array([140,  70], dtype=int64))
x_interest features: [[0.44098206 0.50206612 0.5707804  0.48648649 0.48610121 0.18930164
  0.34515017]]
Prediction x_interest: [1]



### (a) Génération et Test


In [2]:

# 1. Generate Counterfactual
print("Generating Counterfactual...")
cf = whatif.generate_whatif(x_interest=x_interest, model=model, dataset=X_train)

if cf is not None:
    print("Counterfactual found!")
    print("Comparison (Interest vs CF):")
    feature_names = dataset.get_input_labels()
    
    # Create DataFrame for nice display
    df_compare = pd.DataFrame(data=np.vstack([x_interest, cf]), columns=feature_names, index=["x_interest", "Counterfactual"])
    display(df_compare)
    
    # Show diff
    diff = cf - x_interest
    df_diff = pd.DataFrame(data=diff, columns=feature_names, index=["Diff"])
    display(df_diff)

    print("Prediction CF:", model.predict(cf))
else:
    print("No counterfactual found.")

# 2. Evaluate Minimality
if cf is not None:
    print("\nEvaluating Minimality...")
    non_minimal = whatif.evaluate_counterfactual(counterfactual=cf, x_interest=x_interest, model=model)
    print("Indices of non-minimal features:", non_minimal)
    if non_minimal:
        print("Non-minimal features names:", [feature_names[i] for i in non_minimal])


Generating Counterfactual...
Counterfactual found!
Comparison (Interest vs CF):


,area,perimeter,compactness,length,width,asymmetry,groove
x_interest,0.440982,0.502066,0.570780,0.486486,0.486101,0.189302,0.34515
Counterfactual,0.473088,0.557851,0.452813,0.525338,0.467569,0.254834,0.60709


,area,perimeter,compactness,length,width,asymmetry,groove
Diff,0.032106,0.055785,-0.117967,0.038851,-0.018532,0.065532,0.26194


Prediction CF: [0]

Evaluating Minimality...
Indices of non-minimal features: [0, 1, 2, 3, 4, 5]
Non-minimal features names: ['area', 'perimeter', 'compactness', 'length', 'width', 'asymmetry']


### (b) Analyse des attributs et interprétation du contre-factuel

#### Attributs satisfaits

1. **Validité**  
Le contre-factuel généré est valide car il entraîne un **changement effectif de prédiction** :  
- Prédiction de l’observation originale (*x_interest*) : **classe 1**  
- Prédiction du contre-factuel : **classe 0**

---

2. **Plausibilité (Manifold Closeness)**  
La plausibilité est **totalement satisfaite**, car le contre-factuel correspond à une **observation réelle existante dans le dataset**.  
Il respecte donc les corrélations naturelles entre les variables et ne crée pas de point irréaliste.

---

3. **Parcimonie (Sparsity)**  
La parcimonie **n’est pas satisfaite**.  
Plusieurs caractéristiques ont été modifiées simultanément pour obtenir le changement de classe, ce qui est confirmé par l’analyse de minimalité montrant que plusieurs variables sont **non minimales** (*area, perimeter, compactness, length, width, asymmetry*).

---

#### Avantages
- Validité garantie.
- Plausibilité parfaite (pas de point hors distribution).
- Méthode simple et robuste.

---

#### Inconvénients
- Faible parcimonie (beaucoup de changements simultanés).
- Dépend fortement de la densité du dataset.
- Risque de divulgation d’informations (le contre-factuel est un point réel du jeu de données).

---

### Questions d’interprétation

#### 1. Comment le point contre-factuel modifie-t-il les caractéristiques par rapport à l’observation originale ?
Le contre-factuel modifie **plusieurs caractéristiques simultanément**.  
Les principales variations observées sont :
- augmentation de *area*, *perimeter*, *length* et *asymmetry* ;
- diminution de *compactness* et *width* ;
- forte augmentation de *groove*.

Ces changements permettent au modèle de basculer de la classe 1 vers la classe 0.

---

#### 2. Quelle(s) caractéristique(s) a/ont le plus changé pour atteindre la prédiction souhaitée ?
La caractéristique ayant le **plus grand changement absolu** est **groove** (+0.26194).  
Elle apparaît comme le facteur le plus fortement modifié pour inverser la prédiction, suggérant une influence importante sur la décision du modèle.

---

#### 3. Que peut-on conclure sur la sensibilité du modèle à certaines caractéristiques ?
Le fait que le changement de classe nécessite des modifications importantes sur certaines variables, notamment **groove**, suggère que le modèle est **particulièrement sensible** à ces caractéristiques.  
Cependant, comme plusieurs variables changent simultanément, cette approche ne permet pas d’isoler parfaitement l’effet marginal d’une seule caractéristique.  
L’identification de caractéristiques **non minimales** indique que certaines variables modifiées n’étaient pas strictement nécessaires au changement de prédiction.
